In [1]:
import os

In [2]:
%pwd

'/home/vikasrathod/vikasr/Courses_vikas_data/CV_Project/End-to-End-DeepLearningproject/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/vikasrathod/vikasr/Courses_vikas_data/CV_Project/End-to-End-DeepLearningproject'

In [5]:
# Update the entity

from dataclasses import dataclass # decorator 
from pathlib import Path

# this is custom entity we created
# now we are accessing different varible from other files(config/config.py) 
# in below class
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# this class only return these values not other than these 

In [6]:
#pip install python-box

In [7]:
from src.objectClassification.constants import *
from src. objectClassification.utils.common import read_yaml, create_directories



In [8]:
#### Update the configuration manager in src config

In [12]:
class ConfigurationManager:
    # check in side constant directory see __init__ constructor
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH):

        self.config = read_yaml(config_filepath) #return Congif dictionary
        self.params = read_yaml(params_filepath) #return Congif dictionary
        
        # create_directory calling from common.py
        create_directories([self.config.artifacts_root]) 
        


    # check inside config directory check config.yaml 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion # this comming from config.yaml
        print(">>>>>>>>>>>>>>>>>.",config)
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
config = ConfigurationManager()
#config.config.root_dir

[2024-08-26 10:08:48,026: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-26 10:08:48,028: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-26 10:08:48,028: INFO: common: created directory at: artifacts]


In [14]:
d = config.get_data_ingestion_config()

>>>>>>>>>>>>>>>>>. {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1OOcaQ4Ep_eqmyV3hVOKJsoS_zX_EN8fp/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}
[2024-08-26 10:08:48,860: INFO: common: created directory at: artifacts/data_ingestion]


In [15]:
# Data Ingestion Pipeline

import os
import zipfile
import gdown
from src.objectClassification import logger
from src.objectClassification.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config # we will access all config params one by one

    
    def download_file(self)-> str: # type: ignore
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            

            # To get Unique file ID from gdrive
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
       
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [13]:
#config.get_data_ingestion_config()

In [16]:
#Pipeline 
try:
    config = ConfigurationManager()
    print("000000000000000000")
    data_ingestion_config = config.get_data_ingestion_config()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",data_ingestion_config)
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-26 10:08:55,864: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-26 10:08:55,865: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-26 10:08:55,866: INFO: common: created directory at: artifacts]
000000000000000000
>>>>>>>>>>>>>>>>>. {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1OOcaQ4Ep_eqmyV3hVOKJsoS_zX_EN8fp/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}
[2024-08-26 10:08:55,867: INFO: common: created directory at: artifacts/data_ingestion]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://drive.google.com/file/d/1OOcaQ4Ep_eqmyV3hVOKJsoS_zX_EN8fp/view?usp=sharing', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')
[2024-08-26 10:08:55,867: INFO: 1542983971: Downloading data from https://drive.google.com/file/d/1OOcaQ4Ep_e

Downloading...
From: https://drive.google.com/uc?/export=download&id=1OOcaQ4Ep_eqmyV3hVOKJsoS_zX_EN8fp
To: /home/vikasrathod/vikasr/Courses_vikas_data/CV_Project/End-to-End-DeepLearningproject/artifacts/data_ingestion/data.zip
100%|██████████| 20.0M/20.0M [00:03<00:00, 5.55MB/s]

[2024-08-26 10:09:06,173: INFO: 1542983971: Downloaded data from https://drive.google.com/file/d/1OOcaQ4Ep_eqmyV3hVOKJsoS_zX_EN8fp/view?usp=sharing into file artifacts/data_ingestion/data.zip]
